In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

In [2]:
training_data = MNIST(
    root='data/',
    train=True,
    download=True,
    transform=ToTensor(),
)
print(len(training_data))

60000


In [3]:
test_data = MNIST(
    root='data/',
    train=False,
    download=True,
    transform=ToTensor(),
)
print(len(test_data))

10000


In [4]:
batch_size = 80

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([80, 1, 28, 28])
Shape of y: torch.Size([80]) torch.int64


In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten() #conversion en vecteurs unidimensionnels
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 784),
            nn.ReLU(),
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512,10)
    )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=784, bias=True)
    (1): ReLU()
    (2): Linear(in_features=784, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
epochs = 15
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302977  [   80/60000]
loss: 0.260328  [ 8080/60000]
loss: 0.403437  [16080/60000]
loss: 0.190716  [24080/60000]
loss: 0.319627  [32080/60000]
loss: 0.114368  [40080/60000]
loss: 0.137915  [48080/60000]
loss: 0.128319  [56080/60000]
Test Error: 
 Accuracy: 95.5%, Avg loss: 0.137012 

Epoch 2
-------------------------------
loss: 0.056002  [   80/60000]
loss: 0.112917  [ 8080/60000]
loss: 0.111863  [16080/60000]
loss: 0.068857  [24080/60000]
loss: 0.099473  [32080/60000]
loss: 0.058219  [40080/60000]
loss: 0.040977  [48080/60000]
loss: 0.047397  [56080/60000]
Test Error: 
 Accuracy: 95.5%, Avg loss: 0.155637 

Epoch 3
-------------------------------
loss: 0.032231  [   80/60000]
loss: 0.035273  [ 8080/60000]
loss: 0.058215  [16080/60000]
loss: 0.029815  [24080/60000]
loss: 0.087429  [32080/60000]
loss: 0.035250  [40080/60000]
loss: 0.012793  [48080/60000]
loss: 0.025084  [56080/60000]
Test Error: 
 Accuracy: 97.7%, Avg loss: 0.077463 

Epoc